<a href="https://colab.research.google.com/github/ucaokylong/facial_expression_with_fer2013_data/blob/main/fer_2013_EfficientNetB1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/drive')
!cp /path/to/dataset/on/your/drive

In [ ]:
!unzip /content/drive/MyDrive/fer_2013.zip

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os

In [ ]:
fer2013_path = './fer_2013' # Đường dẫn đến folder dataset
train_fer2013_path = os.path.join(fer2013_path, 'train') # Đường dẫn đến folder dataset train
val_fer2013_path = os.path.join(fer2013_path, 'val') # Đường dẫn đến folder dataset val

BATCH_SIZE = 32
IMG_HEIGHT = 197
IMG_WIDTH = 197
RANDOM_SEED = 1

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_fer2013_path, # Đường dẫn đến bộ train
    seed=RANDOM_SEED, # Cài đặt random seed cho việc shuffle và các phép transform (nếu có)
    image_size=(IMG_HEIGHT, IMG_WIDTH), # Cài đặt kích thước ảnh
    shuffle=True, # Bật chế độ shuffle dataset
    batch_size=BATCH_SIZE, # Cài đặt batch size
    color_mode='grayscale' # Đọc ảnh theo định dạng ảnh mức xám
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    val_fer2013_path, # Đường dẫn đến bộ val
    seed=RANDOM_SEED, # Cài đặt random seed cho các phép transform (nếu có)
    image_size=(IMG_HEIGHT, IMG_WIDTH), # Cài đặt kích thước ảnh
    batch_size=BATCH_SIZE, # Cài đặt batch size
    color_mode='grayscale' # Đọc ảnh theo định dạng ảnh mức xám
)

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
    tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset = -1)
])

In [ ]:
base_model = tf.keras.applications.EfficientNetB1(input_shape = (197,197,3),
                                                  include_top = False,
                                                  weights = 'imagenet')

In [ ]:
print(len(base_model.layers))

# Freeze some layers
batch_norm_indices = [2, 4, 6, 9, 13, 14, 18, 21, 24, 26, 28, 31, 34, 38, 41, 43, 45, 46, 53, 56, 60, 63, 66, 70, 73,
                      76, 80, 83, 85, 87, 88, 92, 95, 98, 102, 105, 108, 110, 112, 115, 118, 120, 122, 125, 128, 132,
                      135, 138, 142, 145, 147, 149, 150, 152, 154, 157, 160, 164, 167, 170, 173, 176, 179, 180, 183,
                      185, 187, 190, 194, 198, 202, 205, 208, 210, 212, 215, 218, 220, 222, 225, 228, 232, 235, 238,
                      240, 242, 245, 249, 250, 254, 257, 260, 264, 267, 270, 273, 276, 280, 283, 287, 290, 294, 298,
                      302, 305, 308, 312, 315, 318, 322, 325, 328, 332, 335, 338]


In [ ]:
for i in range(340):
    if i not in batch_norm_indices:
        base_model.layers[i].trainable = False

print(len(base_model.layers))

In [ ]:
global_max = tf.keras.layers.GlobalMaxPool2D()

prediction_layer = tf.keras.layers.Dense(7, activation = 'softmax' )

inputs =tf.keras.Input(shape = (197,197,3))
x = data_augmentation(inputs)
x = base_model(x)
x = tf.keras.layers.BatchNormalization()(x)
x = global_max(x)

x = tf.keras.layers.Dense(132, activation = 'relu')(x)
x = tf.keras.layers.Dense(132, activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(64, activation = 'relu')(x)
x = tf.keras.layers.Dense(64, activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(32, activation = 'relu')(x)
x = tf.keras.layers.Dense(32, activation = 'relu')(x)
x = tf.keras.layers.Dense(32, activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(16, activation = 'relu')(x)
x = tf.keras.layers.Dense(16, activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(16, activation = 'relu')(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(8, activation = 'relu')(x)


outputs = prediction_layer(x)

model = tf.keras.Model(inputs, outputs)


In [ ]:
model.summary()

In [ ]:
# Cấu hình các tham số tối ưu cho việc đọc dữ liệu
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# Khai báo một số giá trị siêu tham số
input_shape = (IMG_HEIGHT, IMG_WIDTH)
EPOCHS = 30
LR = 1e-4

In [ ]:
from keras import regularizers
from keras.callbacks import ModelCheckpoint, CSVLogger, TensorBoard, EarlyStopping, ReduceLROnPlateau

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [ ]:
# Cấu hình một số thông tin cho mô hình

lrd = ReduceLROnPlateau(monitor = 'val_accuracy',patience = 3,verbose = 1,factor = 0.5, min_lr = 0.00001)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LR), # Sử dụng optimizer Adam
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True), # Sử dụng hàm loss SparseCategorialCrossEntropy
    metrics=['accuracy'] # Sử dụng thêm độ đô đánh giá Accuracy
)

In [ ]:
# Thực hiện huấn luyện
history = model.fit( 
    train_ds, # Huấn luyện với bộ train_ds
    validation_data=val_ds, # Đánh giá trên bộ val_ds
    epochs=EPOCHS, # Huấn luyện với số lần lặp = EPOCHS
    callbacks = [lrd]
)

In [ ]:
model.save_weights('test2_fer2013_EffB1.h5')

In [ ]:
from google.colab import files
files.download('test_fer2013_EffB1.h5')

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['training', 'validation'])
plt.title('Loss')
plt.xlabel('epoch')


In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['training','validation'])
plt.title('Accuracy')
plt.xlabel('epoch')